In [1]:
from itertools import combinations_with_replacement
from ehrhart_polynomial import secondary_fan, ehrhart_polynomial, create_polyhedron_from_matrix

In [71]:
from sage.functions.other import factorial
from sage.modules.free_module_element import free_module_element
from sage.rings.polynomial.polynomial_ring_constructor import PolynomialRing

In [3]:
from inspect import *

In [4]:
def generate_cone_points(cone, number):
    dimension = cone._dim
    amb_dim = cone._lattice._Module_free_ambient__degree

    # print(f"cone of dim = {dimension} in {amb_dim}d ambient space")

    if dimension == 0:
        return [(0,)*amb_dim], 0

    vectors = [free_module_element(list(ray)) for ray in cone._rays]

    points = []
    points_len = 0
    index = 1
    while points_len <= number:
        new_points = [sum(combi) for combi in combinations_with_replacement(vectors, index)]
        points += new_points
        index += 1
        points_len += len(new_points)
    return points[points_len-number:]

In [11]:
A = Matrix([[-1, 0], [0, -1], [1, 1], [0, 1]])
sec_fan = secondary_fan(A)
cones = sec_fan["CONES"]
cones

[0-d cone in 1-d lattice N,
 1-d cone in 4-d lattice N,
 1-d cone in 4-d lattice N,
 1-d cone in 4-d lattice N,
 2-d cone in 4-d lattice N,
 2-d cone in 4-d lattice N]

In [6]:
R = PolynomialRing(QQ, "x", 4)
R.gens()

(x0, x1, x2, x3)

In [7]:
f = lambda w, x, y, z: w**2 + x*y + z
R.interpolation(2, f)

x0^2 + x1*x2 + x3

In [57]:
def get_terms_of_order(polynomials, order):
    return [poly.coefficients()[order].constants()[0] if poly.degree() >= order else 0 for poly in polynomials]

In [76]:
def get_term_dict(A_matrix, points):
    polynomials = []
    for point in points:
        polytope = create_polyhedron_from_matrix(A_matrix, point)
        ehr_poly = ehrhart_polynomial(polytope.Vrepresentation(), True)
        polynomials.append(ehr_poly)

    max_degree = max(poly.degree() for poly in polynomials)
    terms = {d: get_terms_of_order(polynomials, d) for d in range(max_degree + 1)}
    return terms

In [78]:
def _compute_piecewise(A_matrix, sec_fan):
    num_variables = sec_fan["DIM"]
    max_degree = sec_fan["LINEALITY_DIM"]

    R = PolynomialRing(QQ, "x", 4)
    S = PolynomialRing(R, "k")
    k = S.gen()

    cone_polys = []
    for cone in sec_fan["CONES"][1:]:
        estimated_period = 1
        needed_points = factorial(num_variables + max_degree)//( factorial(num_variables)*factorial(max_degree) )
        cone_points = generate_cone_points(cone, needed_points)
        term_dict = get_term_dict(A_matrix, cone_points)

        cone_poly = 0
        for degree, terms in term_dict.items():
            cone_poly += R.interpolation(max_degree, cone_points, terms)*k**degree
        print(cone_poly)
        cone_polys.append(cone_poly)
    return cone_polys
        

In [79]:
_compute_piecewise(A, sec_fan)

1
21/8*x3^2*k^2 + 13/4*x3*k + 1
9/2*x3^2*k^2 - 11/2*x3*k + 1


ValueError: Could not find a solution.

In [63]:
cone_points

[(-2, -2, 4, 6),
 (-3, -3, 6, 9),
 (-4, -4, 8, 12),
 (-5, -5, 10, 15),
 (-6, -6, 12, 18),
 (-7, -7, 14, 21),
 (-8, -8, 16, 24),
 (-9, -9, 18, 27),
 (-10, -10, 20, 30),
 (-11, -11, 22, 33),
 (-12, -12, 24, 36),
 (-13, -13, 26, 39),
 (-14, -14, 28, 42),
 (-15, -15, 30, 45),
 (-16, -16, 32, 48)]

In [70]:
cone_poly = 0
R = PolynomialRing(QQ, "x", 4)
S = PolynomialRing(R, "k")
k = S.gen()
for degree, terms in term_dict.items():
    cone_poly += R.interpolation(2, cone_points, terms)*k**degree
print(cone_poly)

1
